# Set up the class fundementals 

In [1]:
# import os, sys
# import collections
import numpy as _np
# import matplotlib.markers as markers
# import matplotlib.pyplot as plt
# import timeit
# import collections
# from scipy.stats import median_abs_deviation as mad
# import multiprocessing
# import cProfile
# import itertools
from numba import jit as _jit
from numba import njit as _njit
from bed_reader import open_bed as _open_bed
# import warnings
# warnings.filterwarnings('ignore') # this is just to hide all the warnings
# import rpy2.robjects as robjects
# import matplotlib.pyplot as plt # change font globally to Times 
# plt.style.use('ggplot')
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Times New Roman",
#     "font.sans-serif": ["Times New Roman"],
#     "font.size": 12})

# os.chdir(sys.path[0]) # ensure working direcotry is set same as the file

/home/Snoopy/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
######################################  some SCAD and MCP things  #######################################
@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def soft_thresholding(x, lambda_):
    '''
    To calculate soft-thresholding mapping of a given ONE-DIMENSIONAL tensor, BESIDES THE FIRST TERM (so beta_0 will not be penalized). 
    This function is to be used for calculation involving L1 penalty term later. 
    '''
    return _np.hstack((_np.array([x[0]]), _np.where(_np.abs(x[1:])>lambda_, x[1:] - _np.sign(x[1:])*lambda_, 0)))

soft_thresholding(_np.random.rand(20),3.1)

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD(x, lambda_, a=3.7):
    '''
    To calculate SCAD penalty value;
    #x can be a multi-dimensional tensor;
    lambda_, a are scalars;
    Fan and Li suggests to take a as 3.7 
    '''
    # here I notice the function is de facto a function of absolute value of x, therefore take absolute value first to simplify calculation 
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, lambda_*x, _np.where(x<a*lambda_, (2*a*lambda_*x - x**2 - lambda_**2)/(2*(a - 1)), lambda_**2 * (a+1)/2))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_grad(x, lambda_, a=3.7):
    '''
    To calculate the gradient of SCAD wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    # here decompose x to sign and its absolute value for easier calculation
    sgn = _np.sign(x)
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, lambda_*sgn, _np.where(x<a*lambda_, (a*lambda_*sgn-sgn*x)/(a-1), 0))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP(x, lambda_, gamma):
    '''
    To calculate MCP penalty value; 
    #x can be a multi-dimensional tensor. 
    '''
    # the function is a function of absolute value of x 
    x = _np.abs(x)
    temp = _np.where(x<=gamma*lambda_, lambda_*x - x**2/(2*gamma), .5*gamma*lambda_**2)
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_grad(x, lambda_, gamma):
    '''
    To calculate MCP gradient wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    temp = _np.where(_np.abs(x)<gamma*lambda_, lambda_*_np.sign(x)-x/gamma, _np.zeros_like(x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_concave(x, lambda_, a=3.7):
    '''
    The value of concave part of SCAD penalty; 
    #x can be a multi-dimensional tensor. 
    '''
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, 0., _np.where(x<a*lambda_, (lambda_*x - (x**2 + lambda_**2)/2)/(a-1), (a+1)/2*lambda_**2 - lambda_*x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_concave_grad(x, lambda_, a=3.7):
    '''
    The gradient of concave part of SCAD penalty wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    sgn = _np.sign(x)
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, 0., _np.where(x<a*lambda_, (lambda_*sgn-sgn*x)/(a-1), -lambda_*sgn))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_concave(x, lambda_, gamma):
    '''
    The value of concave part of MCP penalty; 
    #x can be a multi-dimensional tensor. 
    '''
    # similiar as in MCP
    x = _np.abs(x)
    temp = _np.where(x<=gamma*lambda_, -(x**2)/(2*gamma), (gamma*lambda_**2)/2 - lambda_*x)
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_concave_grad(x, lambda_, gamma):
    '''
    The gradient of concave part of MCP penalty wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    temp = _np.where(_np.abs(x) < gamma*lambda_, -x/gamma, -lambda_*_np.sign(x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 


# Implementation

In [3]:
# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _SNP_update_smooth_grad_convex_LM(N, SNP_ind, bed, beta_md, y, outcome_iid):
    '''
    Update the gradient of the smooth convex objective component.
    '''
    p=len(list(bed.sid))
    gene_iid = _np.array(list(bed.iid))
    _y = y[_np.intersect1d(outcome_iid, gene_iid, assume_unique=True, return_indices=True)[1]]
    gene_ind = _np.intersect1d(gene_iid, outcome_iid, assume_unique=True, return_indices=True)[1]
    # first calcualte _=X@beta_md-y
    _ = _np.zeros(N)
    for j in SNP_ind:
        _X = bed.read(_np.s_[:,j], dtype=_np.int8).flatten()
        _X = _X[gene_ind] # get gene iid also in outcome iid
        _ += _X*beta_md[j+1] # +1 because intercept
    _ += beta_md[0] # add the intercept
    _ -= _y
    # then calculate _XTXbeta = X.T@X@beta_md = X.T@_
    _XTXbeta = _np.zeros(p)
    for j in SNP_ind:
        _X = bed.read(_np.s_[:,j], dtype=_np.int8).flatten()
        _X = _X[gene_ind] # get gene iid also in outcome iid
        _XTXbeta[j] = _X@_
    _XTXbeta = _np.hstack((_np.array([np.sum(_)]),_XTXbeta))
    del _
    return 1/N*_XTXbeta

# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _SNP_update_smooth_grad_SCAD_LM(N, SNP_ind, bed, beta_md, y, outcome_iid, _lambda, a):
    '''
    Update the gradient of the smooth objective component for SCAD penalty.
    '''
    return _SNP_update_smooth_grad_convex_LM(N=N, SNP_ind=SNP_ind, bed=bed, beta_md=beta_md, y=y, outcome_iid=outcome_iid) + SCAD_concave_grad(x=beta_md, lambda_=_lambda, a=a)

# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _SNP_update_smooth_grad_MCP_LM(N, SNP_ind, bed, beta_md, y, outcome_iid, _lambda, gamma):
    '''
    Update the gradient of the smooth objective component for MCP penalty.
    '''
    return _SNP_update_smooth_grad_convex_LM(N=N, SNP_ind=SNP_ind, bed=bed, beta_md=beta_md, y=y, outcome_iid=outcome_iid) + MCP_concave_grad(x=beta_md, lambda_=_lambda, gamma=gamma)


# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _SNP_lambda_max_LM(bed, y, outcome_iid, N, SNP_ind):
    """
    Calculate the lambda_max, i.e., the minimum lambda to nullify all penalized betas.
    """
#     X_temp = X.copy()
#     X_temp = X_temp[:,1:]
#     X_temp -= _np.mean(X_temp,0).reshape(1,-1)
#     X_temp /= _np.std(X_temp,0)
#     y_temp = y.copy()
#     y_temp -= _np.mean(y)
#     y_temp /= _np.std(y)
    p=len(list(bed.sid))
    grad_at_0 = _SNP_update_smooth_grad_convex_LM(N=N, SNP_ind=SNP_ind, bed=bed, beta_md=_np.zeros(p), y=y, outcome_iid=outcome_iid)
    return lambda_max



# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SNP_UAG_LM_SCAD_MCP(bed_file, bim_file, fam_file, outcome, outcome_iid, SNP_ind, L_convex, beta_0 = _np.ones(1), tol=1e-2, maxit=500, _lambda=.5, penalty="SCAD", a=3.7, gamma=2.):
    '''
    Carry out the optimization for penalized LM for a fixed lambda.
    '''
    bed = _open_bed(filepath=bed_file, fam_filepath=fam_file, bim_filepath=bim_file)
    y = outcome
    p = len(list(bed.sid))
    gene_iid = _np.array(list(bed.iid))
    N = len(_np.intersect1d(outcome_iid, gene_iid, assume_unique=True, return_indices=True)[1])
    if _np.all(beta_0==_np.ones(1)):
        _ = _np.zeros(p)
        _y = y[_np.intersect1d(outcome_iid, gene_iid, assume_unique=True, return_indices=True)[1]]
        _y -= _np.mean(_y)
        for j in SNP_ind:
            _X = bed.read(_np.s_[:,j], dtype=_np.int8).flatten()
            _X = _X[gene_ind] # get gene iid also in outcome iid
            _X -= _np.mean(_X)
            _[j] = _X@_y
        beta = _np.sign(_)
        beta = _np.hstack((np.array([0]), beta))
    else:
        beta = beta_0
    # passing other parameters
    smooth_grad = _np.ones(p+1)
    beta_ag = beta.copy()
    beta_md = beta.copy()
    k = 0
    converged = False
    opt_alpha = 1.
    old_speed_norm = 1.
    speed_norm = 1.
    restart_k = 0
    
    if penalty == "SCAD":
#         L = _np.max(_np.array([L_convex, 1./(a-1)]))
        L = _np.linalg.norm(_np.array([L_convex, 1./(a-1)]), ord=_np.infty)
        opt_beta = .99/L
        while ((not converged) or (k<3)) and k <= maxit:
            k += 1
            if old_speed_norm > speed_norm and k - restart_k>=3: # in this case, restart
                opt_alpha = 1. # restarting
                restart_k = k # restarting
            else: # restarting
                opt_alpha = 2/(1+(1+4./opt_alpha**2)**.5) #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound 
            opt_lambda = opt_beta/opt_alpha #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound
            beta_md_old = beta_md.copy() # restarting
            beta_md = (1-opt_alpha)*beta_ag + opt_alpha*beta
            old_speed_norm = speed_norm # restarting
            speed_norm = _np.linalg.norm(beta_md - beta_md_old, ord=2) # restarting
            converged = (_np.linalg.norm(beta_md - beta_md_old, ord=_np.infty) < tol)
            smooth_grad = _SNP_update_smooth_grad_SCAD_LM(N=N, SNP_ind=SNP_ind, bed=bed, beta_md=beta_md, y=y, outcome_iid=outcome_iid, _lambda=_lambda, a=a)
            beta = soft_thresholding(x=beta - opt_lambda*smooth_grad, lambda_=opt_lambda*_lambda)
            beta_ag = soft_thresholding(x=beta_md - opt_beta*smooth_grad, lambda_=opt_beta*_lambda)
#             converged = _np.all(_np.max(_np.abs(beta_md - beta_ag)/opt_beta) < tol).item()
#             converged = (_np.linalg.norm(beta_md - beta_ag, ord=_np.infty) < (tol*opt_beta))
    else:
#         L = _np.max(_np.array([L_convex, 1./(gamma)]))
        L = _np.linalg.norm(_np.array([L_convex, 1./(gamma)]), ord=_np.infty)
        opt_beta = .99/L
        while ((not converged) or (k<3)) and k <= maxit:
            k += 1
            if old_speed_norm > speed_norm and k - restart_k>=3: # in this case, restart
                opt_alpha = 1. # restarting
                restart_k = k # restarting
            else: # restarting
                opt_alpha = 2/(1+(1+4./opt_alpha**2)**.5) #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound 
            opt_lambda = opt_beta/opt_alpha #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound
            beta_md_old = beta_md.copy() # restarting
            beta_md = (1-opt_alpha)*beta_ag + opt_alpha*beta
            old_speed_norm = speed_norm # restarting
            speed_norm = _np.linalg.norm(beta_md - beta_md_old, ord=2) # restarting
            converged = (_np.linalg.norm(beta_md - beta_md_old, ord=_np.infty) < tol)
            smooth_grad = _SNP_update_smooth_grad_MCP_LM(N=N, SNP_ind=SNP_ind, bed=bed, beta_md=beta_md, y=y, outcome_iid=outcome_iid, _lambda=_lambda, gamma=gamma)
            beta = soft_thresholding(x=beta - opt_lambda*smooth_grad, lambda_=opt_lambda*_lambda)
            beta_ag = soft_thresholding(x=beta_md - opt_beta*smooth_grad, lambda_=opt_beta*_lambda)
#             converged = _np.all(_np.max(_np.abs(beta_md - beta_ag)/opt_beta) < tol).item()
#             converged = (_np.linalg.norm(beta_md - beta_ag, ord=_np.infty) < (tol*opt_beta))
    return k, beta_md


# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SNP_solution_path_LM(bed_file, bim_file, fam_file, outcome, outcome_iid, lambda_, L_convex, SNP_ind, beta_0 = _np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2.):
    '''
    Carry out the optimization for the solution path without the strong rule.
    '''
    bed = _open_bed(filepath=bed_file, fam_filepath=fam_file, bim_filepath=bim_file)
    p = len(list(bed.sid))
    beta_mat = _np.zeros((len(lambda_)+1, p+1))
    for j in range(len(lambda_)): 
        beta_mat[j+1,:] = SNP_UAG_LM_SCAD_MCP(bed_file=bed_file, bim_file=bim_file, fam_file=fam_file, outcome=outcome, SNP_ind=SNP_ind, L_convex=L_convex, beta_0 = beta_mat[j,:], tol=tol, maxit=maxit, _lambda=lambda_[j], penalty=penalty, outcome_iid=outcome_iid, a=a, gamma=gamma)[1]
    return beta_mat[1:,:]



# Testing

In [4]:
import numpy as np
from bed_reader import open_bed

bed_file = r"./sim/sim1.bed"
bim_file = r"./sim/sim1.bim"
fam_file = r"./sim/sim1.fam"

_bed = open_bed(filepath=bed_file, fam_filepath=fam_file, bim_filepath=bim_file)
outcome = np.random.rand(_bed.iid_count)
outcome_iid = _bed.iid
true_beta = np.array([12.,-15.,16.])

# here since the plink files are very small, I just use Python to calculate L_convex -- normally it should be calculated using other softwares, e.g., flashpca 
temp = np.zeros((_bed.iid_count, _bed.sid_count))
for j in np.arange(_bed.sid_count):
    temp[:,j] = _bed.read(_np.s_[:,j], dtype=_np.int8).flatten()
L_convex = 1/_bed.iid_count*(_np.linalg.eigvalsh(temp@temp.T)[-1])
print("L_convex is:", L_convex)

for j in np.arange(3):
    outcome += true_beta[j]*_bed.read(_np.s_[:,j], dtype=_np.int8).flatten()

iid_ind = np.random.permutation(np.arange(_bed.iid_count))

outcome = outcome[iid_ind]
outcome_iid = outcome_iid[iid_ind]

print(SNP_solution_path_LM(bed_file=bed_file, 
                            bim_file=bim_file, 
                            fam_file=fam_file, 
                            outcome=outcome, 
                            outcome_iid=outcome_iid, 
                            lambda_=np.linspace(.001,10,20)[::-1],
                            SNP_ind=np.arange(10), 
                            L_convex=L_convex, 
                            beta_0 = np.ones(1),
                            tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2.)[:,:10])


L_convex is: 951.1723835414942
[[ 4.68810205e+00  2.78015194e+00  0.00000000e+00  3.29606170e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.69746444e+00  2.77748954e+00  0.00000000e+00  3.29608707e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.70680489e+00  2.77636314e+00  0.00000000e+00  3.29764527e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.71611455e+00  2.77676152e+00  0.00000000e+00  3.30072520e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.72538466e+00  2.77867357e+00  0.00000000e+00  3.30531587e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 4.73460652e+00  2.78208830e+00  0.00000000e+00  3.31140639e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  